In [ ]:
### word2vec 속도개선 
--- 
- 기존 문제점 

    - 입력층의 원핫표현과 가중치 행렬 W의 곱 계산
    - 은닉층의 가중치 행렬 W의 곱및 softmax계층의 계산 
---

- Embedding 도입 


In [1]:
# embedding 구현
import numpy as np

class Embedding:
    def __init__(self,W):
        self.params=[W]
        self.grads=[np.zeros_like(W)]
        self.idx=None
    def forward(self,idx):
        W =self.params
        self.idx=idx
        out=W[idx]
        return out

    # def backward(self,dout):
    #     dw, =self.grads
    #     dw[...]=0 #->dw가 0이 되는 것이 아니라 그 형태를 0으로 덮어씌우는 작업 사실상 np.zeros로도가능할듯
    #     dw[self.idx]=dout
    #     return None
    def backward(self,dout):
        dw,= self.grads
        dw[...]=0
        for i , word_id in enumerate(self.idx):
            dw[word_id]+=dout[i]
            #혹은 
            #np.add.at(dw,self.idx,dout)-> 이작업은 for문 자체를 대신하는 것이다 numpy의 특성중 하나로 for문을 쓰지않고 한번에 연산을 가능하게 해주는것 ㅍㅍ
        return None
## 하지만 이 backward 과정에는 문제가 있다. dh-> dw를 갱신할떄 즉 weight값을 갱신해줄떄 만약에 처음 idx가 
# 0,2,4,0 처럼 2개의 값을 갖는 경우는 기울기가 중복되어 덮어 씌어지게 된다 이를 해결하기 위해서 덮어쓰지 않고 dh의 각 행의 값을 dw의 해당행에 더해준다 .

### Note
---
` 다중 분류 문제를 이진분류로 다루려면 '정답'과 '오답' 각각에 대해 바르게 분류할 수 있어야 한다. 따라서 
긍정적 예와 부정적 예 모두를 대상으로 학습해야 한다`

- 따라서 모든 부정적 답에 대해서 결론을 주는 것이 아니라 5~6개의 부정적 예를 샘플링해 사용한다.`네거티브샘플링`

In [2]:
##embedding class 구현

class EmbeddingDot:
    def __intit__(self,W):
        self.embed=Embedding(W)
        self.params=self.embed.params
        self.grads=self.embed.grads
        self.cache=None
    
    def forward(self,h,idx):
        target_W=self.embed.forward(W)
        out=np.sum(target_W*h,axis=1)
        self.cache=(h,target_W)
        return out

    def backward(self,dout):
        h,target_W=self.cache
        dout=dout.reshape(dout.shape[0],1)

        dtarget_W=dout*h
        self.embed.backward(dtarget_W)
        dh=dout*target_W
        return dh

In [ ]:
# Negative sampling loss 구하기

class NegativeSamplingLoss:
    def __init__(self,W,corpus,power=0.75,sample_size=5):
        self.sample_size=sample_size
        self.sampler=UnigramSampler(corpus,power,sample_size)
        self.loss_layer=[SigmoidWithLoss() for  _ in range(sample_size+1)]
        self.embed_dot_layers=[EmbeddingDot(W) for _ in range(sample_size+1)]
        self.params,self.grad=[],[]
        for layers in self.embed_dot_layers:
            self.params+=layers.params
            self.grads+=layers.grads


#chapter 4 요약

- Embedding 계층은 단어의 분산 표현을 담고 있으며, 순전파 시 지정한 단어 ID의 벡터를 추출한다.

- word2vec은 어휘 수의 증가에 비례하여 계산량도 증가하므로 근사치로 계산하는 빠른 기법을 사용한다.

- 네거티브 샘플링은 부정적 예를 몇개를 샘플링하는 기법으로 이를 이용하면 다중 분류를 이진 분류처럼 취급할 수 있다. 
- word2vec으로 얻은 단어의 분산 표현에는 단어의 의미가 녹아들어 있으며, 비슷한 맥락에서 사용되는 단어는 단어 벡터 공간에서 가까이 위치한다.

- word2ve의 단어의 분산 표현을 이용하면 유추문제를 벡터의 덧셈과 뺄셈으로 풀 수있게된다.

- word2vec 은 전이 학습 측면에서 특히 중요하며, 그 단어의 분산 표현은 다양한 자연어 처리 작업에 이용 할 수있다.